More At : https://github.com/iSiddharth20/DeepLearning-ImageClassification-Toolkit

In [ ]:
'''
Working Directories
Name of Model
'''
import os

# Directory where Pickle Files will be Stored
PICKLE_DIR = '../PickleFiles/'

# Directory where Models will be Stored (Folder will be Creted By Code)
MODEL_DIR = '../TrainedModels/'
os.makedirs(os.path.dirname(MODEL_DIR), exist_ok=True)

# Name of Model Used
MODEL_NAME = 'CNN_ResNet50'

In [ ]:
'''
Importing Necessary Libraries and Packages
'''
# Disable TensorFlow Warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Helpers
import pickle

# To log Trainnig Time
from datetime import datetime

# Importing TensorFlow
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
'''
Importing the following One-Hot-Encoded Data from Pickle Files
    - ClassLabels_train
    - ClassLabels_test
    - ClassLabels_validation
Importing the following from Pickle Files
    - ImgArray_train
    - ImgArray_test
    - ImgArray_validation
Importing One-Hot-Encoding
'''

# Exporting ClassLabels_train
try:
    with open(PICKLE_DIR+'ClassLabels_train.pkl', 'rb') as f: ClassLabels_train = pickle.load(f)
    print('Import Successful for : ClassLabels_train.pkl ')  
except:
    print('Import Unsuccessful for : ClassLabels_train.pkl ')

# Exporting ClassLabels_test
try:
    with open(PICKLE_DIR+'ClassLabels_test.pkl', 'rb') as f: ClassLabels_test = pickle.load(f)
    print('Import Successful for : ClassLabels_test.pkl ')  
except:
    print('Import Unsuccessful for : ClassLabels_test.pkl ')

# Exporting ClassLabels_validation
try:
    with open(PICKLE_DIR+'ClassLabels_validation.pkl', 'rb') as f: ClassLabels_validation = pickle.load(f)
    print('Import Successful for : ClassLabels_validation.pkl ')  
except:
    print('Import Unsuccessful for : ClassLabels_validation.pkl ')

    
# Exporting ImgArray_train
try:
    with open(PICKLE_DIR+'ImgArray_train.pkl', 'rb') as f: ImgArray_train = pickle.load(f)
    print('Import Successful for : ImgArray_train.pkl')  
except:
    print('Import Unsuccessful for : ImgArray_train.pkl ')

# Exporting ImgArray_test
try:
    with open(PICKLE_DIR+'ImgArray_test.pkl', 'rb') as f: ImgArray_test = pickle.load(f)
    print('Import Successful for : ImgArray_test.pkl')
except:
    print('Import Unsuccessful for : ImgArray_test.pkl ') 

# Exporting ImgArray_validation
try:
    with open(PICKLE_DIR+'ImgArray_validation.pkl', 'rb') as f: ImgArray_validation = pickle.load(f)
    print('Import Successful for : ImgArray_validation.pkl')
except:
    print('Import Unsuccessful for : ImgArray_validation.pkl ') 

    
# Exporting One-Hot-Encoding 
try:
    with open(PICKLE_DIR+'OHE.pkl', 'rb') as f: OHE = pickle.load(f)
    print('Import Successful for : OHE.pkl')
except:
    print('Import Unsuccessful for : OHE.pkl ') 

In [ ]:
'''
Preparing Data Pipeline
    - Convert All Data to TensorFlow Format
    - Defining Data Augmentation Function
    - Defining Rescaled Image Height, Width

Change BATCH_SIZE as per Requirement.
    - Higher the BATCH_SIZE (usually 16-32) the better
    - If you get 'Resource Exhaust Error', decrease the BATCH_SIZE
'''

BATCH_SIZE = 8 
IMG_HEIGHT = 400
IMG_WIDTH = 600
IMG_CHANNELS = 3 # Assuming RGB Image

# Training Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((ImgArray_train,ClassLabels_train))
train_dataset = train_dataset.shuffle(10000, reshuffle_each_iteration=True)
train_dataset = train_dataset.cache()
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
del(ImgArray_train) # Clear RAM

# Testing Dataset
test_dataset = tf.data.Dataset.from_tensor_slices((ImgArray_test,ClassLabels_test))
test_dataset = test_dataset.batch(BATCH_SIZE)
del(ImgArray_test) # Clear RAM
del(ClassLabels_test) # Clear RAM

# Validation Dataset
val_dataset = tf.data.Dataset.from_tensor_slices((ImgArray_validation,ClassLabels_validation))
val_dataset = val_dataset.batch(BATCH_SIZE)
del(ImgArray_validation) # Clear RAM

# Data Augmentation
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
  tf.keras.layers.experimental.preprocessing.RandomTranslation(0.03, 0.06),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.015),
  tf.keras.layers.experimental.preprocessing.RandomHeight(0.1, interpolation='nearest'),
  tf.keras.layers.experimental.preprocessing.RandomWidth(0.1), 
])

# Steps Per Epoch and Validation Steps
steps_per_epoch = len(ClassLabels_train) // BATCH_SIZE
validation_steps = len(ClassLabels_validation) // BATCH_SIZE
del(ClassLabels_train) # Clear RAM
del(ClassLabels_validation) # Clear RAM

In [ ]:
# Define Callbacks
earlystopper = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, verbose=2)
checkpointer = tf.keras.callbacks.ModelCheckpoint(MODEL_DIR+MODEL_NAME+'.h5', save_best_only=True ,verbose=1)

# Learning Rate Scheduler (Change as per Requirement)
initial_learning_rate = 0.001
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min', cooldown=1, min_lr=1e-5)

In [ ]:
'''
Building Training Model [ResNet50]
    - Load Base Model
    - Build Final Model
    - Compile Final Model
'''

# Loading Base Model
OHE_classes = OHE.categories_[0]
num_classes = len(OHE_classes)
del(OHE) # Clear RAM
base_model = tf.keras.applications.resnet50.ResNet50(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), include_top=False, 
    weights='imagenet', classes=num_classes
)

# Building Final Model
inputs = tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
x = data_augmentation(inputs)
x = base_model(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

# Compiling Final Model
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.SGD(learning_rate=initial_learning_rate, momentum=0.9),
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), AUC(name='auc')])

# View the Model Summary
model.summary()

In [ ]:
'''
Train the Model
'''
begin = datetime.now()

EPOCHS = 50 # Change as per Requirement
history = model.fit(train_dataset, validation_data=val_dataset, 
                    steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, 
                    epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1, 
                    callbacks = [checkpointer, earlystopper, lr_scheduler])

finish = datetime.now()

In [ ]:
'''
Display Time Required (In Seconds) to Train the Model
'''

total_time = finish-begin
print('Total Training Time (In Seconds) for '+MODEL_NAME+' : ',total_time.total_seconds())

In [ ]:
'''
Evaluate the Model
'''

model = tf.keras.models.load_model(MODEL_DIR+MODEL_NAME+'.h5')

print('Training : ', model.evaluate(train_dataset, steps=steps_per_epoch))
print('Validation : ', model.evaluate(val_dataset))
print('Testing : ', model.evaluate(test_dataset))